In [ ]:
import numpy as np

# Initialize global counter
fun_call = 0

def obj_piping(D):
    """
    Objective function for piping optimization.
    Updates the global call counter.
    """
    global fun_call
    fun_call += 1

    # Constants
    L = 1000.0  # ft
    Q = 20.0    # gpm

    # Calculate head loss (hp)
    # 4.4e-8*(L*Q^3)/(D^5) + 1.92e-9*(L*Q^2.68)/(D^4.68)
    term1 = 4.4e-8 * (L * Q**3) / (D**5)
    term2 = 1.92e-9 * (L * Q**2.68) / (D**4.68)
    hp = term1 + term2

    # Calculate Cost (y)
    # 0.45*L + 0.245*L*D^1.5 + 3.25*(hp)^0.5 + 61.6*(hp)^0.925 + 102
    y = (0.45 * L +
         0.245 * L * D**1.5 +
         3.25 * (hp)**0.5 +
         61.6 * (hp)**0.925 +
         102)

    return y

def one_dim_pw(xx, s, op2_func):
    """
    Powell's Quadratic Interpolation Method
    """
    dela = 0.005
    alp0 = 0.01

    # Initialize lists (size 3)
    alpha = [0.0] * 3
    y = [0.0] * 3

    # --- Point 1 ---
    alpha[0] = alp0
    x1 = xx + alpha[0] * s
    y[0] = op2_func(x1)

    # --- Point 2 ---
    alpha[1] = alpha[0] + dela
    x2 = xx + alpha[1] * s
    y[1] = op2_func(x2)

    # --- Bracketing Logic (Point 3) ---
    if y[1] >= y[0]:
        alpha[2] = alpha[0] - dela
    else:
        alpha[2] = alpha[0] + 2 * dela

    eps = 100.0
    delta = 100.0

    # Initialize best_alpha for scope safety
    best_alpha = alpha[1]

    # Loop until convergence
    while eps > 0.001 or delta > 0.001:
        # Evaluate Point 3
        x3 = xx + s * alpha[2]
        y[2] = op2_func(x3)

        # --- Quadratic Interpolation Step ---
        a0, a1, a2 = alpha[0], alpha[1], alpha[2]
        f0, f1, f2 = y[0], y[1], y[2]

        # Denominator
        denom = 2 * ((a1 - a2)*f0 + (a2 - a0)*f1 + (a0 - a1)*f2)

        if denom == 0:
            break # Avoid division by zero

        # Numerator
        num = (a1**2 - a2**2)*f0 + (a2**2 - a0**2)*f1 + (a0**2 - a1**2)*f2

        # Optimal alpha prediction (Vertex of parabola)
        alpha_opt = num / denom

        # --- Calculate convergence metrics ---
        # Find index of the best point so far
        best_idx = np.argmin(y)
        best_alpha = alpha[best_idx]
        best_y = y[best_idx]

        # eps: change in the variable (alpha)
        eps = abs(alpha_opt - best_alpha)

        # Evaluate function at new optimal prediction
        x_opt = xx + s * alpha_opt
        y_opt = op2_func(x_opt)

        # delta: change in function value
        delta = abs(best_y - y_opt)

        # --- Update Points for Next Iteration ---
        # Replace the worst point (highest function value) with the new optimal point
        worst_idx = np.argmax(y)
        alpha[worst_idx] = alpha_opt
        y[worst_idx] = y_opt

        # Update return value to the current best
        final_best_idx = np.argmin(y)
        best_alpha = alpha[final_best_idx]

    return best_alpha

# --- Main Execution ---

# Reset global counter
fun_call = 0

# Parameters
x0 = 0.25
x_end = 6.0
l = x_end - x0

# Execute Optimization
# Passing 'obj_piping' function object directly
al_opt = one_dim_pw(x0, l, obj_piping)

# Calculate final Diameter D
D = x0 + al_opt * l

# Print Results
print(f"al_opt   = {al_opt:.4f}")
print(f"D        = {D:.4f}")
print(f"fun_call = {fun_call}")